Problem Statement:

Here are working with two dataframes:

Transactions Dataset: Contains customer transactions with information about transaction type (debit or credit) and the transaction amount.

Amounts Dataset: Contains the current balance for each customer.
Your task is to adjust the current balances in the Amounts Dataset based on the transactions from the Transactions Dataset. Specifically:

For debit transactions, subtract the transaction amount from the current balance.
For credit transactions, add the transaction amount to the current balance.
If a customer has no transactions, their current balance remains unchanged.

In [0]:
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    FloatType,
)

# Define the schema for transactions
transaction_schema = StructType(
    [
        StructField("customer_id", IntegerType(), True),
        StructField("transaction_type", StringType(), True),
        StructField("transaction_amount", FloatType(), True),
    ]
)

# Create the transactions DataFrame
transactions_data = [
    (1, "credit", 30.0),
    (1, "debit", 90.0),
    (2, "credit", 50.0),
    (3, "debit", 57.0),
    (2, "debit", 90.0),
]

transactions_df = spark.createDataFrame(transactions_data, schema=transaction_schema)

# Show the transactions DataFrame
transactions_df.display()

# Define the schema for amounts
amount_schema = StructType(
    [
        StructField("customer_id", IntegerType(), True),
        StructField("current_amount", FloatType(), True),
    ]
)

# Create the amounts DataFrame
amounts_data = [(1, 1000.0), (2, 2000.0), (3, 3000.0), (4, 4000.0)]

amounts_df = spark.createDataFrame(amounts_data, schema=amount_schema)

# Show the amounts DataFrame
amounts_df.display()

customer_id,transaction_type,transaction_amount
1,credit,30.0
1,debit,90.0
2,credit,50.0
3,debit,57.0
2,debit,90.0


customer_id,current_amount
1,1000.0
2,2000.0
3,3000.0
4,4000.0


In [0]:
from pyspark.sql.functions import *

transaction_agg = (
    transactions_df.withColumn(
        "transaction_amount",
        when(col("transaction_type") == "debit", -col("transaction_amount")).otherwise(
            col("transaction_amount")
        ),
    )
    .groupBy("customer_id")
    .agg(sum("transaction_amount").alias("total_transaction_amount"))
)
transaction_agg.display()

customer_id,total_transaction_amount
1,-60.0
2,-40.0
3,-57.0


In [0]:
final_df = amounts_df.join(transaction_agg, on="customer_id", how="left")
final_df.display()

customer_id,current_amount,total_transaction_amount
1,1000.0,-60.0
2,2000.0,-40.0
3,3000.0,-57.0
4,4000.0,null


In [0]:
final_df = final_df.withColumn(
    "current_amount",
    when(
        col("total_transaction_amount").isNotNull(),
        col("current_amount") + col("total_transaction_amount"),
    ).otherwise(col("current_amount")),
).drop("total_transaction_amount")

final_df.display()

customer_id,current_amount
1,940.0
2,1960.0
3,2943.0
4,4000.0


In [0]:
# Create the DataFrames as temporary SQL tables
transactions_df.createOrReplaceTempView("transactions")
amounts_df.createOrReplaceTempView("amounts")

In [0]:
# Write a SQL query to adjust the current amount based on transactions
result_df = spark.sql(
    """
    SELECT a.customer_id,
           a.current_amount + COALESCE(SUM(CASE 
               WHEN t.transaction_type = 'debit' THEN -t.transaction_amount
               WHEN t.transaction_type = 'credit' THEN t.transaction_amount
               ELSE 0
           END), 0) AS updated_amount
    FROM amounts a
    LEFT JOIN transactions t
    ON a.customer_id = t.customer_id
    GROUP BY a.customer_id, a.current_amount
"""
)

# Show the final result
result_df.display()

customer_id,updated_amount
1,940.0
2,1960.0
3,2943.0
4,4000.0


Explanation:

Step 1: Both DataFrames (transactions_df and amounts_df) are registered as temporary SQL views (transactions and amounts).

Step 2: We perform a LEFT JOIN to combine the two tables on customer_id.
A CASE statement is used to adjust the transaction_amount based on whether the transaction_type is 'debit' or 'credit'.
Debits are subtracted, and credits are added to the current_amount.
We use COALESCE() to handle cases where there are no transactions for a customer.

Step 3: The SUM function calculates the total impact of the transactions, and this is added to the current_amount to compute the updated_amount.